In [42]:
import tensorflow as tf
from onnx_tf.frontend import tensorflow_graph_to_onnx_model

In [55]:
import os, argparse

# The original freeze_graph function
# from tensorflow.python.tools.freeze_graph import freeze_graph 

#dir = os.path.dirname(os.path.realpath(__file__))

def freeze_graph(model_dir, output_node_names):
    """Extract the sub graph defined by the output nodes and convert 
    all its variables into constant 
    Args:
        model_dir: the root folder containing the checkpoint state file
        output_node_names: a string, containing all the output node's names, 
                            comma separated
    """
    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exist. Please specify an export "
            "directory: %s" % model_dir)

    if not output_node_names:
        print("You need to supply the name of a node to --output_node_names.")
        return -1
    
    # Prints all nodes in graph for debugging
    #[print(n.name) for n in tf.get_default_graph().as_graph_def().node]

    # We retrieve our checkpoint fullpath
    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path
    
    # We precise the file fullname of our freezed graph
    absolute_model_dir = "/".join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + "/frozen_model.pb"

    # We clear devices to allow TensorFlow to control on which device it will load operations
    clear_devices = True

    # We start a session using a temporary fresh Graph
    with tf.Session(graph=tf.Graph()) as sess:
        # We import the meta graph in the current default Graph
        saver = tf.train.import_meta_graph(input_checkpoint + '.meta', clear_devices=clear_devices)

        # We restore the weights
        saver.restore(sess, input_checkpoint)

        # We use a built-in TF helper to export variables to constants
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess, # The session is used to retrieve the weights
            tf.get_default_graph().as_graph_def(), # The graph_def is used to retrieve the nodes 
            output_node_names.split(",") # The output node names are used to select the useful nodes
        ) 

        # Finally we serialize and dump the output graph to the filesystem
        with tf.gfile.GFile('models/iris/' + output_graph, "wb") as f:
            f.write(output_graph_def.SerializeToString())
        print("%d ops in the final graph." % len(output_graph_def.node))

    #return output_graph_def

freeze_graph("models\iris", "dnn/head/predictions/class_ids,dnn/head/predictions/probabilities") # Runs freezer function

INFO:tensorflow:Restoring parameters from models\iris\model.ckpt-4000
INFO:tensorflow:Froze 6 variables.
INFO:tensorflow:Converted 6 variables to const ops.
78 ops in the final graph.


In [58]:
# Converts frozen model to ONNX format
with tf.gfile.GFile("models/iris/frozen_model.pb", "rb") as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    onnx_model = tensorflow_graph_to_onnx_model(graph_def,
                                     "dnn/head/predictions/class_ids", #This is the output node
                                     #opset=6,
                                     graph_name="graph",
                                     ignore_unimplemented=True)

    file = open("models/iris/graph.onnx", "wb")
    file.write(onnx_model.SerializeToString())
    file.close()

c:\users\camp-pry\appdata\local\continuum\anaconda3\envs\tensorflow_iris\lib\site-packages\onnx_tf\frontend.py:533: UserWarning: RandomShuffleQueueV2 op is not implemented.
  exception(message)


ValueError: You passed in an iterable attribute but I cannot figure out its applicable type.